In [1]:
import pandas as pd
dataset=pd.read_csv('cardio_dataset.csv').values
data=dataset[:,0:7]
target=dataset[:,7]
target=target.reshape(-1,1)
#data=data.reshape(-1,1)
print(target[:5])
print(data[:,0:7])

[[1.1]
 [7. ]
 [7. ]
 [0.4]
 [2.2]]
[[  2.  48. 236. ...   0.   2.   0.]
 [  1.  48. 260. ...   0.   2.   1.]
 [  1.  44. 187. ...   1.   2.   0.]
 ...
 [  1.  65. 212. ...   1.   1.   0.]
 [  1.  66. 184. ...   0.   1.   0.]
 [  2.  45. 203. ...   0.   2.   0.]]


In [2]:
from sklearn.preprocessing import MinMaxScaler
data_scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=data_scaler.fit_transform(data)
target_scaler=MinMaxScaler(feature_range=(0,1))
scaled_target=target_scaler.fit_transform(target)


In [3]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(scaled_data,scaled_target,test_size=0.2)

def create_model(layers,activation):
    model=Sequential()
    for i,nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=7))
            model.add(Activation(activation))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
    model.add(Dense(1)) #Note: No activation beyond this point
    
    model.compile(optimizer='adam',loss= 'mse',metrics=["accuracy"])
    return model
model= KerasClassifier(build_fn=create_model,verbose=0)


Using TensorFlow backend.


In [4]:
model

In [5]:
layers=[[32],[32,32],[64,32,16,8]]
activations=['sigmoid','relu']
param_grid =dict(layers=layers,activation=activations,epochs=[1000])
grid=GridSearchCV(estimator=model,param_grid=param_grid)


In [6]:
grid_result =grid.fit(train_data,train_target)

In [9]:
[grid_result.best_score_,grid_result.best_params_]

[0.04082784536360349,
 {'activation': 'relu', 'epochs': 1000, 'layers': [64, 32, 16, 8]}]

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['loss'],'r',label='training_loss')
plt.plot(history.history['val_loss'],'g',label='val loss')
plt.xlabel('# epoch')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
result=model.predict(test_data)

from sklearn.metrics import r2_score

r2=r2_score(test_target,result)
print('r2 score:',r2)

In [ ]:
target_scaler.fit(target)

real_test_target=target_scaler.inverse_transform(test_target)
real_result=target_scaler.inverse_tarnsform(result)

print('ActualTargets',real_test_target[:10.T)
print('Predictedtargets,real_result[:10].T)